In [1]:
!pip install transformers --quiet
!pip install "ray[tune]" torch torchvision pytorch-lightning --quiet
!pip install -e .

Obtaining file:///D:/02_Studium/SBX/mad-recime/network/LSTM/pl
  Attempting uninstall: nlp-utils
    Found existing installation: nlp-utils 0.0.1
    Uninstalling nlp-utils-0.0.1:
      Successfully uninstalled nlp-utils-0.0.1
  Running setup.py develop for nlp-utils


In [2]:
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive/')
except:
    # local exec
    dataPath = os.path.join(os.getcwd(), '../../../data/2022_03_19')
    logDir = os.path.join(os.getcwd(), 'logs/title/')
    saveDir = os.path.join(os.getcwd(), 'model/title/')
    rayDir = os.path.join(os.getcwd(), 'ray/title/')

    sys.path.append(os.getcwd())
else: 
    rootDir = '/content/drive/MyDrive/'

    TIMESTAMP = '2022_03_30'

    dataPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP
    logDir = rootDir + 'Colab Notebooks/recime/LSTM/pl/logs/title/'
    saveDir = rootDir + 'Colab Notebooks/recime/LSTM/pl/model/title/'
    rayDir = rootDir + 'Colab Notebooks/recime/LSTM/pl/ray/title/'

    sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM/pl')

if not os.path.exists(logDir):
  os.makedirs(logDir)
if not os.path.exists(rayDir):
  os.makedirs(rayDir)
if not os.path.exists(saveDir):
  os.makedirs(saveDir)

In [3]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import loggers as pl_loggers

import torch
import numpy as np
import re
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback,  TuneReportCheckpointCallback

In [4]:
import nlp_utils.model as models
import nlp_utils.data_module as data_module

import importlib

In [5]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
num_gpus = len(available_gpus)
accelerator = 'gpu' if num_gpus > 0 else 'cpu'
accelerator, num_gpus

('cpu', 0)

In [6]:
# set seed to get consistent results, deactivate if random results are wanted
seed_everything(42)

Global seed set to 42


42

In [7]:
importlib.reload(data_module)

titleMod = data_module.TitleDataModule(dataPath, setSize=8)
titleMod.setup()

d:\02_Studium\SBX\mad-recime\network\LSTM\pl\nlp_utils\preProc.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))
d:\02_Studium\SBX\mad-recime\network\LSTM\pl\nlp_utils\preProc.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))


In [8]:
importlib.reload(models)

from nlp_utils.config import create_config

tb_logger = pl_loggers.TensorBoardLogger(save_dir=logDir)

config = create_config({})

config['vocabSize'] = titleMod.vocab_size

model = models.EmbedLSTM(config)

In [10]:
trainer = Trainer(accelerator=accelerator, devices=-1, max_epochs=config['epochs'], logger=tb_logger)
trainer.fit(model, titleMod)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\Hannes\miniconda3\envs\madRecime\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name      | Type             | Params
-----------------------------------------------
0 | embed     | Embedding        | 171 K 
1 | lstm      | LSTM             | 571 K 
2 | linear    | Linear           | 147 K 
3 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
889 K     Trainable params
0         Non-trainable params
889 K     Total params
3.560     Total estimated model params size (MB)
C:\Users\Hannes\miniconda3\envs\madRecime\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:631: UserWarning: Checkpoint directory d:\02_St

Epoch 11:   0%|          | 0/100 [00:18<?, ?it/s, loss=2.25, v_num=2]

# Test

In [ ]:
# mnist = MNISTDataModule(my_path)
# model = LitClassifier()

# trainer = Trainer()
# trainer.fit(model, mnist)

# Optimization

In [14]:
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.05,
    patience=3,
    verbose=False,
    mode="min",
    divergence_threshold=3.00,
)

class MyPrintingCallback(Callback):
    def on_fit_start(self, trainer, pl_module):
        print("Starting to train!")

    def on_fit_end(self, trainer, pl_module):
        print("Finished training")

    def on_test_start(self, trainer, pl_module):
        print("Start to test")

    def on_test_end(self, trainer, pl_module):
        print("Finished testing")


# checkpoint_callback = ModelCheckpoint(
#     monitor="val_epoch_stance_F1",
#     filename="{epoch}-{val_loss:.2f}-{val_epoch_stance_F1:.2f}",
#     save_top_k=3,
#     mode="max",
# )

callback = TuneReportCallback(
    {"loss": "val_loss"}, on="validation_end"
)


# training loop that tests out different hyperparameters and saves the results into the log folder
def train_tune(config, callbacks, epochs=100, acc='cpu'):
    dataModule = data_module.TitleDataModule(dataPath, setSize=8, num_workers=1, config=config)
    dataModule.setup()
    
    config["vocabSize"] = dataModule.vocab_size
    
    model = models.EmbedLSTM(config)

    trainer = pl.Trainer(
        accelerator=acc,
        devices=-1,
        log_every_n_steps=1,
        flush_logs_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        default_root_dir=logDir,
        max_epochs=epochs,
    )  

    trainer.fit(model, datamodule=dataModule)

    # might not be called due to scheduler and reporter which cancel training early if results don't look promising
    # trainer.test(model, datamodule=dataModule)


# config with radom sampling for learning rate and batch size
config = {
    'lr': tune.sample_from(lambda: 10 ** np.random.uniform(-5, -3)),
    'batchSize': tune.choice([1, 8, 16, 32, 64, 128]),
    'hiddenSize': tune.choice([16, 32, 64, 128, 256, 512, 1024]),
    'epochs': 30,
    'num_trials': 1,
}


# callbacks = [MyPrintingCallback(), early_stop_callback]
callbacks = [callback]

scheduler = ASHAScheduler(max_t=config['epochs'], grace_period=1, reduction_factor=2)

reporter = CLIReporter(
    parameter_columns=['lr', 'batchSize', 'hiddenSize'],
    metric_columns=['loss'],
)

# ray.init(local_mode=True, num_cpus=4, num_gpus=0)  # for debugging

# create versioning for multiple runs
# def atoi(text):
#     return int(text) if text.isdigit() else text


# def natural_keys(text):
#     return [atoi(c) for c in re.split("(\d+)", text)]


# log_dir = "../logs/StancePrediction_SemEval/lightning_logs/"
# log_path = os.path.join(path, log_dir)
# os.makedirs(os.path.dirname(log_path), exist_ok=True)
# ver = os.listdir(os.path.join(path, log_dir))
# ver.sort(key=natural_keys)
# if ver:
#     version = int(ver[-1].split("_", 2)[-1]) + 1
# else:
#     version = 0

In [15]:
# start hyperparameter optimization
analysis = tune.run(
    tune.with_parameters(
        train_tune, callbacks=callbacks, epochs=config['epochs'], acc=accelerator
    ),
    config=config,
    num_samples=config['num_trials'],
    local_dir=rayDir,
    name='version_ReciMe',
    metric='loss',
    mode='min',
    scheduler=scheduler,
    progress_reporter=reporter,
    resources_per_trial={'cpu': 2, 'gpu': num_gpus}
)

# get some information from the optimization
best_trial = analysis.best_trial  # Get best trial
best_config = analysis.best_config  # Get best trial's hyperparameters
best_logdir = analysis.best_logdir  # Get best trial's logdir
best_checkpoint = analysis.best_checkpoint  # Get best trial's best checkpoint
best_result = analysis.best_result  # Get best trial's last results
best_result_df = analysis.best_result_df  # Get best result as pandas dataframe

# Get a dataframe with the last results for each trial
df_results = analysis.results_df

# Get a dataframe of results for a specific score or mode
df = analysis.dataframe(metric="loss", mode="min")
# df2 = analysis.dataframe(metric="val_epoch_F1", mode="max") # check how to include multiple metrics


print("Best hyperparameters found were: ", analysis.best_config)

== Status ==
Current time: 2022-04-08 23:38:07 (running for 00:00:00.15)
Memory usage on this node: 6.2/7.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/4 CPUs, 0/1 GPUs, 0.0/1.42 GiB heap, 0.0/0.71 GiB objects
Result logdir: d:\02_Studium\SBX\mad-recime\network\LSTM\pl\ray\title\version_ReciMe
Number of trials: 1/1 (1 PENDING)
+------------------------+----------+-------+-------------+-------------+--------------+
| Trial name             | status   | loc   |          lr |   batchSize |   hiddenSize |
|------------------------+----------+-------+-------------+-------------+--------------|
| train_tune_2da3e_00000 | PENDING  |       | 0.000157513 |           8 |           64 |
+------------------------+----------+-------+-------------+-------------+--------------+


== Status ==
Current time: 2022-04-08 23:38:12 (running for 00:00:05.15)
Memory usage on this node: 6.